# Mitigating Bias

In [ ]:
!pip install BlackBoxAuditing
!pip install aif360

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394752 sha256=af2a0d151ac99c632669ae3e6623e7934ff43bf41bf9e070c0aac7f23e3551de
  Stored in directory: /root/.cache/pip/wheels/c0/4f/b1/80e1b0790df07536470758fe0a4f9ff8fa942fd9fe30bbb192
Successfully built BlackBoxAuditing
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 3.6 MB/s eta 0:00:00


### Loading and preparing the dataset

In [ ]:
# First, read-in the data and check for null values
import numpy as np
import pandas as pd
import aif360
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
pd.options.mode.chained_assignment = None  # default='warn', silencing Setting With Copy warning
df = pd.read_csv('credit_risk.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban,Y
977,LP002975,Male,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban,Y
978,LP002980,Male,No,0,Graduate,No,3250,1993.0,126.0,360.0,NaN,Semiurban,Y
979,LP002986,Male,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural,N


In [ ]:
# See the different columns and check for null entries
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            981 non-null    object 
 1   Gender             957 non-null    object 
 2   Married            978 non-null    object 
 3   Dependents         956 non-null    object 
 4   Education          981 non-null    object 
 5   Self_Employed      926 non-null    object 
 6   ApplicantIncome    981 non-null    int64  
 7   CoapplicantIncome  981 non-null    float64
 8   LoanAmount         954 non-null    float64
 9   Loan_Amount_Term   961 non-null    float64
 10  Credit_History     902 non-null    float64
 11  Property_Area      981 non-null    object 
 12  Loan_Status        981 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 99.8+ KB


In [ ]:
# Remove rows with null values
df = df.dropna(how='any', axis = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769 entries, 1 to 980
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            769 non-null    object 
 1   Gender             769 non-null    object 
 2   Married            769 non-null    object 
 3   Dependents         769 non-null    object 
 4   Education          769 non-null    object 
 5   Self_Employed      769 non-null    object 
 6   ApplicantIncome    769 non-null    int64  
 7   CoapplicantIncome  769 non-null    float64
 8   LoanAmount         769 non-null    float64
 9   Loan_Amount_Term   769 non-null    float64
 10  Credit_History     769 non-null    float64
 11  Property_Area      769 non-null    object 
 12  Loan_Status        769 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 84.1+ KB


I then want to check to see the breakdown of values for the outcome variable, `Loan_Status`.

In [ ]:
target_counts = df['Loan_Status'].value_counts()
target_counts
df['Gender'].value_counts()

Male      624
Female    145
Name: Gender, dtype: int64

In [ ]:
# Drop unnecessary column
df = df.drop(['Loan_ID'], axis = 1)

### Encode categorical variables

In [ ]:
# Encode Male as 1, Female as 0
df.loc[df.Gender == 'Male', 'Gender'] = 1
df.loc[df.Gender == 'Female', 'Gender'] = 0
# Encode Y Loan_Status as 1, N Loan_Status as 0
df.loc[df.Loan_Status == 'Y', 'Loan_Status'] = 1
df.loc[df.Loan_Status == 'N', 'Loan_Status'] = 0
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,1,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,1,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,1,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1
5,1,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...
975,1,Yes,1,Graduate,No,2269,2167.0,99.0,360.0,1.0,Semiurban,1
976,1,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban,1
977,1,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban,1
979,1,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural,0


In [ ]:
y = df['Loan_Status']
y=y.astype('int')
y

1      0
2      1
3      1
4      1
5      1
      ..
975    1
976    1
977    1
979    0
980    1
Name: Loan_Status, Length: 769, dtype: int64

In [ ]:
# Replace the categorical values with the numeric equivalents that we have above
categoricalFeatures = ['Property_Area', 'Married', 'Dependents', 'Education', 'Self_Employed']
# Iterate through the list of categorical features and one hot encode them.
for feature in categoricalFeatures:
    onehot = pd.get_dummies(df[feature], prefix=feature)
    df = df.drop(feature, axis=1)
    df = df.join(onehot)
df

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
1,1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,0,1,0,1,0,0,1,0,1,0
2,1,3000,0.0,66.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,0,1
3,1,2583,2358.0,120.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,0,1,1,0
4,1,6000,0.0,141.0,360.0,1.0,1,0,0,1,1,0,1,0,0,0,1,0,1,0
5,1,5417,4196.0,267.0,360.0,1.0,1,0,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1,2269,2167.0,99.0,360.0,1.0,1,0,1,0,0,1,0,1,0,0,1,0,1,0
976,1,4009,1777.0,113.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,1
977,1,4158,709.0,115.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,1,0
979,1,5000,2393.0,158.0,360.0,1.0,0,1,0,0,0,1,1,0,0,0,1,0,1,0


### Separate dataset by x and y

In [ ]:
from sklearn.model_selection import train_test_split
encoded_df = df.copy()
x = df.drop(['Loan_Status'], axis = 1)

### Create Test and Train splits

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_std = scaler.fit_transform(x)
# We will follow an 80-20 split pattern for our training and test data, respectively
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

### Calculating actual disparate impact and Statistical Parity Difference on testing values from original dataset
Disparate Impact is defined as the ratio of favorable outcomes for the unpriviliged group divided by the ratio of favorable outcomes for the priviliged group.
The acceptable threshold is between .8 and 1.25, with .8 favoring the priviliged group, and 1.25 favoring the unpriviliged group.

In [ ]:
actual_test = x_test.copy()
actual_test['Loan_Status_Actual'] = y_test
actual_test.shape

binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=actual_test,
    label_names=['Loan_Status_Actual'],
    protected_attribute_names=['Gender'])


bldm = BinaryLabelDatasetMetric(binaryLabelDataset, unprivileged_groups=[{"Gender": 0}], privileged_groups=[{"Gender": 1}])

print("Statistical Parity Difference:", bldm.statistical_parity_difference())
print("Disparate Impact Ratio:", bldm.disparate_impact())

Statistical Parity Difference: -0.12268907563025211
Disparate Impact Ratio: 0.8302325581395349


In [ ]:
# Priviliged group: Males (1)
# Unpriviliged group: Females (0)
male_df = actual_test[actual_test['Gender'] == 1]
num_of_priviliged = male_df.shape[0]
female_df = actual_test[actual_test['Gender'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [ ]:
unpriviliged_outcomes = female_df[female_df['Loan_Status_Actual'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.6

In [ ]:
priviliged_outcomes = male_df[male_df['Loan_Status_Actual'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.7226890756302521

In [ ]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Loan Status: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Loan Status: 0.8302325581395349


### Training a model on the original dataset

In [ ]:
from sklearn.linear_model import LogisticRegression
# Liblinear is a solver that is very fast for small datasets, like ours
model = LogisticRegression(solver='liblinear', class_weight='balanced')

In [ ]:
model.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', solver='liblinear')

### Evaluating performance

In [ ]:
# Let's see how well it predicted with a couple values
y_pred = pd.Series(model.predict(x_test))
y_test = y_test.reset_index(drop=True)
z = pd.concat([y_test, y_pred], axis=1)
z.columns = ['True', 'Prediction']
z.head()

,True,Prediction
0,1,1
1,1,1
2,0,0
3,0,0
4,0,1


In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
import aif360.sklearn.metrics

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))

Accuracy: 0.8116883116883117
Precision: 0.875
Recall: 0.8504672897196262


### Calculating disparate impact on predicted values by model trained on original dataset

In [ ]:
# We now need to add this array into x_test as a column for when we calculate the fairness metrics.
from aif360.metrics import BinaryLabelDatasetMetric

y_pred = model.predict(x_test)
x_test['Loan_Status_Predicted'] = y_pred
original_output = x_test
original_output

binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=original_output,
    label_names=['Loan_Status_Predicted'],
    protected_attribute_names=['Gender'])


bldm = BinaryLabelDatasetMetric(binaryLabelDataset, unprivileged_groups=[{"Gender": 0}], privileged_groups=[{"Gender": 1}])

print("Statistical Parity Difference:", bldm.statistical_parity_difference())
print("Disparate Impact Ratio:", bldm.disparate_impact())

Statistical Parity Difference: -0.24537815126050416
Disparate Impact Ratio: 0.664367816091954


In [ ]:
# Priviliged group: Males (1)
# Unpriviliged group: Females (0)
male_df = original_output[original_output['Gender'] == 1]
num_of_priviliged = male_df.shape[0]
female_df = original_output[original_output['Gender'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [ ]:
unpriviliged_outcomes = female_df[female_df['Loan_Status_Predicted'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.4857142857142857

In [ ]:
priviliged_outcomes = male_df[male_df['Loan_Status_Predicted'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.7310924369747899

In [ ]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Loan Status: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Loan Status: 0.664367816091954


### Applying the Disparate Impact Remover to the dataset

In [ ]:
# We are going to be using the dataset with categorical features encoded, encoded_df
encoded_df

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes
1,1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,0,1,0,1,0,0,1,0,1,0
2,1,3000,0.0,66.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,0,1
3,1,2583,2358.0,120.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,0,1,1,0
4,1,6000,0.0,141.0,360.0,1.0,1,0,0,1,1,0,1,0,0,0,1,0,1,0
5,1,5417,4196.0,267.0,360.0,1.0,1,0,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1,2269,2167.0,99.0,360.0,1.0,1,0,1,0,0,1,0,1,0,0,1,0,1,0
976,1,4009,1777.0,113.0,360.0,1.0,1,0,0,1,0,1,0,0,0,1,0,1,0,1
977,1,4158,709.0,115.0,360.0,1.0,1,0,0,1,0,1,1,0,0,0,1,0,1,0
979,1,5000,2393.0,158.0,360.0,1.0,0,1,0,0,0,1,1,0,0,0,1,0,1,0


In [ ]:
import aif360
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.metrics import BinaryLabelDatasetMetric

binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=encoded_df,
    label_names=['Loan_Status'],
    protected_attribute_names=['Gender'])

di = DisparateImpactRemover(repair_level = 1.0)
dataset_transf_train = di.fit_transform(binaryLabelDataset)
transformed = dataset_transf_train.convert_to_dataframe()[0]
transformed

,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Loan_Status
1,1.0,3958.0,1483.0,108.0,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,2600.0,0.0,59.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,1.0,2241.0,2333.0,102.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
4,1.0,4723.0,0.0,115.0,360.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
5,1.0,4402.0,3683.0,189.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1.0,2101.0,2183.0,79.0,360.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
976,1.0,3719.0,1762.0,94.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
977,1.0,3762.0,717.0,95.0,360.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
979,1.0,4230.0,2333.0,130.0,360.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


### Train a model using the dataset that underwent the pre-processing

In [ ]:
x_trans = transformed.drop(['Loan_Status'], axis = 1)
y = transformed['Loan_Status']
# Liblinear is a solver that is effective for relatively smaller datasets.
model = LogisticRegression(solver='liblinear', class_weight='balanced')
scaler = StandardScaler()
data_std = scaler.fit_transform(x_trans)
# Splitting into test and training
# We will follow an 80-20 split pattern for our training and test data
x_trans_train,x_trans_test,y_trans_train,y_trans_test = train_test_split(x_trans, y, test_size=0.2, random_state = 0)

In [ ]:
model.fit(x_trans_train, y_trans_train)

LogisticRegression(class_weight='balanced', solver='liblinear')

### Evaluating performance

In [ ]:
# See how well it predicted with a couple values
y_trans_pred = pd.Series(model.predict(x_trans_test))
y_trans_test = y_trans_test.reset_index(drop=True)
z = pd.concat([y_trans_test, y_trans_pred], axis=1)
z.columns = ['True', 'Prediction']
z.head()
# Again, it predicts 4/5 correctly in this sample

,True,Prediction
0,1.0,1.0
1,1.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,1.0


In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_trans_pred))
print("Precision:", metrics.precision_score(y_test, y_trans_pred))
print("Recall:", metrics.recall_score(y_test, y_trans_pred))

Accuracy: 0.8246753246753247
Precision: 0.8846153846153846
Recall: 0.8598130841121495


### Calculating disparate impact on predicted values by model trained on transformed dataset

In [ ]:
# We now need to add this array into x_test as a column for when we calculate the fairness metrics.
y_trans_pred = model.predict(x_trans_test)
x_trans_test['Loan_Status_Predicted'] = y_trans_pred
transformed_output = x_trans_test
transformed_output

binaryLabelDataset = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=transformed_output,
    label_names=['Loan_Status_Predicted'],
    protected_attribute_names=['Gender'])


bldm = BinaryLabelDatasetMetric(binaryLabelDataset, unprivileged_groups=[{"Gender": 0}], privileged_groups=[{"Gender": 1}])

print("Statistical Parity Difference:", bldm.statistical_parity_difference())
print("Disparate Impact Ratio:", bldm.disparate_impact())

Statistical Parity Difference: -0.20840336134453785
Disparate Impact Ratio: 0.7116279069767442


Disparate Impact is defined as the ratio of favorable outcomes for the unpriviliged group divided by the ratio of favorable outcomes for the priviliged group. The acceptable threshold is between .8 and 1.25, with .8 favoring the priviliged group, and 1.25 favoring the unpriviliged group.

In [ ]:
# Priviliged group: Males (1)
# Unpriviliged group: Females (0)
male_df = transformed_output[transformed_output['Gender'] == 1]
num_of_priviliged = male_df.shape[0]
female_df = transformed_output[transformed_output['Gender'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [ ]:
unpriviliged_outcomes = female_df[female_df['Loan_Status_Predicted'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.5142857142857142

In [ ]:
priviliged_outcomes = male_df[male_df['Loan_Status_Predicted'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.7226890756302521

In [ ]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Loan Status: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Loan Status: 0.7116279069767442
